# Basic Transformations for 2D Mobile Robots

This notebook presents fundamental handling of transformations encountered with 2D mobile robots in Python and the Robot Operating System (ROS).
Naming conventions generally adhere to the book *S. Thrun, W. Burgard, and D. Fox, Probabilistic Robotics [(link)](https://mitpress.mit.edu/9780262201629/probabilistic-robotics/)*.

Presented conversions and code examples rely on the NumPy [(link)](https://numpy.org/) and Scipy Spatial [(link)](https://docs.scipy.org/doc/scipy/reference/spatial.html) libraries.

# Table of Concents

* [**Fundamentals**](#fundamentals)
* [**Conventions for 2D Mobile Robots in ROS**](#conventions)
    - [**Conversion to/from Rotation Matrices**](#2drotationmat)
    - [**Conversion to/from ROS Pose**](#2drospose)
* [**Transform Interfacing with ROS**](#rosinterfacing)
    - [**Publishing Transforms**](#tfpublisher)
    - [**Listening to Transforms**](#tflistener)

## Fundamentals <a class="anchor" id="fundamentals"></a>

Coordinate transformations are required in mobile robot programming as they create spatial relations between entities. They transform measurements taken relative to one coordinate frame to be relative to a different coordinate frame. In general, a transformation is depicted by a pose, consisting of a translation $T$ and rotation $R$ component.

The translation component consists of three coordinates, which denote the offset along each axis ($x$, $y$, $z$). There are several representations of rotation in 3D, each with distinct advantages and disadvantages.

* **Euler Angles**: This representation describes rotation as a sequence of three angles, typically denoted as rotations about the $x$, $y$, and $z$ axes (roll, pitch, yaw). Euler angles are intuitive but can suffer from issues like gimbal lock, where certain rotations become indistinguishable.

    $$
    \begin{pmatrix}
    x, y, z
    \end{pmatrix}
    $$

* **Rotation Matrices**: A rotation matrix is a 3x3 matrix that represents rotation in three-dimensional space. This matrix can be applied to vectors to rotate them, and it is often used for combining multiple rotations. Rotation matrices are stable but can be cumbersome for certain operations, such as interpolation. The following example shows a rotation matrix for a 2D rotation of $\theta$ around the $z$ axis.

    $$
    \begin{pmatrix}
    cos(\theta), -sin(\theta), 0\\
    sin(\theta), cos(\theta), 0\\
    0, 0, 1
    \end{pmatrix}
    $$

* **Quaternions**: Quaternions are a four-dimensional representation that encodes rotation as an imaginary number ($x$, $y$, $z$, $w$). Contrary to euler angles, they are unambiguous and they are efficient in interpolating rotations. Furthermore, using quaternion multiplication, rotations can be easily combined. Note that $x$, $y$ and $z$ of quaternions are different from euler angles.

    $$
    \begin{pmatrix}
    x, y, z, w
    \end{pmatrix}
    $$

# Conventions for 2D Mobile Robots in ROS <a class="anchor" id="conventions"></a>

ROS natively uses quaterions to represent rotations, causing transforms to be represented using three values for translation and four values for rotation. However in ROS, 2D mobile robots usually move horizontally on the world's $xy$ plane and rotate around the world's (and the robot's) $z$ axis. Therefore, the robot pose can be simplified to three values: $(x, y, \theta)$ with $x$ and $y$ denoting translation in the $xy$ plane and $\theta$ denoting rotation around the $z$ axis. This representation is also refered to as the **robot state**. $\theta$ is one of the euler angles.

## Conversion to/from Rotation Matrices <a class="anchor" id="2drotationmat"></a>

Since the rotation is performed around the $z$ axis, we can directly substitute the robot pose's components into the rotation matrix.

$$
\begin{pmatrix}
cos(\theta), -sin(\theta), x\\
sin(\theta), cos(\theta), y\\
0, 0, 1
\end{pmatrix}
$$

In oder to convert the matrix back to the robot pose, we can directly substitute the translation components. The rotation is retrieved using $atan2$.

In [1]:
import numpy as np

def pose2transform_matrix(pose):
    x, y, theta = pose
    return np.array([
        [np.cos(theta), -np.sin(theta), x],
        [np.sin(theta), np.cos(theta), y],
        [0, 0, 1]
    ])

def transform_matrix2pose(tf):
    theta = np.arctan2(tf[1,0], tf[0,0])
    return np.array([tf[0,2], tf[1,2], theta])

# Transform pose to matrix and back to demonstrate the conversion
pose = [0, 2, 3.1415]
mat = pose2transform_matrix(pose)
retpose = transform_matrix2pose(mat)

print("Input pose: \n{} \n\nRotation Matrix: \n{} \n\nOutput Pose: \n{}".format(
    pose, mat, retpose
))

Input pose: 
[0, 2, 3.1415] 

Rotation Matrix: 
[[-9.99999996e-01 -9.26535897e-05  0.00000000e+00]
 [ 9.26535897e-05 -9.99999996e-01  2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

Output Pose: 
[0.     2.     3.1415]


Transform matrices can easily be combined to a single transform by performing matrix multiplication. This is natively supported via the @ operator by the NumPy library [(link)](https://numpy.org/) used to create the rotation matrix in the example code.

Let us assume a 2D mobile robot exists relative to a world frame with a goal deafined relative to the robot. The relation can be graphed as

*world $\to$ robot $\to$ goal*

with each arrow describing a transformation. The left arrow denotes the world to robot transform, while the right arrow depicts robot to goal. We can now calculate the goal's pose in world coordinates by converting both poses to transform matrices, performing matrix multiplication and converting them back to the state representation.

In [2]:
world2robot = [0, 2, 3.1415] # world2robot
robot2goal = [2, 1, 1.5708] # robot2goal

world2goal_mat = pose2transform_matrix(world2robot) @ pose2transform_matrix(robot2goal)
world2goal = transform_matrix2pose(world2goal_mat)

# Multiply transforms
print("Goal pose relative to world: \n{}".format(
    world2goal))

Goal pose relative to world: 
[-2.00009265  1.00018531 -1.57088531]


The arrows indicate that transformations are directional. This direction has to be considered when combining transformations. An easy to follow rule of thumb is to only multiply in direction of the arrow as shown in the previous example. Depending on how transforms are provided, it may be required to invert the transforms. This is achieved by inverting the matrix.

Let us assume a similar example as previously, however, now the transformations from world to the robot and from world to the goal are given and the goal pose relative to the robot must be calculated.

*robot $\leftarrow$ world $\to$ goal*

In order to apply matrix multiplication, the robot to world transformation must be inverted as shon in the next example. The example uses NumPy's matrix inversion, whose documentation can be found [(here)](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html). After inversion, the graph allows for the two transform matrices to be multiplied.

*robot $\to$ world $\to$ goal*

In [3]:
world2robot = [0, 2, 3.1415]
world2goal = [-2, 1, -1.5708]

robot2world_mat = np.linalg.inv(pose2transform_matrix(world2robot))
robot2goal_mat = robot2world_mat @ pose2transform_matrix(world2goal)
robot2goal = transform_matrix2pose(robot2goal_mat)

print("Goal pose relative to robot; \n{}".format(
    robot2goal))

Goal pose relative to robot; 
[1.99990734 1.0001853  1.57088531]


## Conversion to/from ROS Pose <a class="anchor" id="2drospose"></a>

In order to convert to and from the quaternions used by ROS to present a pose's rotation component, we use the SciPy Spatial library [(link)](https://docs.scipy.org/doc/scipy/reference/spatial.html). The following example uses ROS' Pose message [(link)](https://docs.ros.org/en/noetic/api/geometry_msgs/html/msg/Pose.html), however the same concept for conversions applies to all transformation representations in ROS that use the *Point* and *Quaternion* messages for translation and roation respectively.

The Point message depicting translation can be directly populated with the robot pose's $x$ and $y$ values and 0 for $z$. However, in order to populate the Pose's Quaternion component, we first instantiate a SciPy rotation instance from a single euler angle ($\theta$ represents the rotation around $z$) and convert that directly to a quaternion.

In [4]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from geometry_msgs.msg import Pose, Point, Quaternion

def pose2ROS(pose):
    p = Pose()
    p.position = Point(x=pose[0], y=pose[1], z=0)
    quat = R.from_euler("z", pose[2]).as_quat() # Euler to quaternion, scipy uses xyzw
    p.orientation = Quaternion(x=quat[0], y=quat[1], z=quat[2], w=quat[3])
    return p

def ROS2pose(Pose_msg):
    quat, point = Pose_msg.orientation, Pose_msg.position
    theta = R.from_quat([quat.x, quat.y, quat.z, quat.w]).as_euler("xyz")[2]
    return np.array([point.x, point.y, theta])

# Transform pose to ROS message and back to demonstrate the conversion
pose = [0, 2, 3.1415]
Pose_msg = pose2ROS(pose)
retpose = ROS2pose(Pose_msg)

print("Input pose: \n{} \n\nRotation Matrix: \n{} \n\nOutput Pose: \n{}".format(
    pose, Pose_msg, retpose
))

Input pose: 
[0, 2, 3.1415] 

Rotation Matrix: 
position: 
  x: 0
  y: 2
  z: 0
orientation: 
  x: 0.0
  y: 0.0
  z: 0.999999998926914
  w: 4.632679487995776e-05 

Output Pose: 
[0.     2.     3.1415]


# Transform Interfacing with ROS <a class="anchor" id="rosinterfacing"></a>

This section provides example code how send and receive transformations to and from the Robot Operating System (ROS). While code using the `tf2_ros` module works in versions 1 and 2 of ROS, ROS node initialisation in the following examples uses ROS 1.

To run the following code examples, make sure that your Jupyter notebook interface is installed in the same Python environment as ROS. Furthermore, a `roscore` must be running in the background.

## Publishing Transforms <a class="anchor" id="tfpublisher"></a>

In ROS, the `tf2_ros` package is used to keep track of multiple coordinate frames over time and to publish transformations between them. Transforms are published by creating a `tf2_ros.TransformBroadcaster` object serving as the connection to ROS' transformation system. The standard message type for transforms is `TransformStamped` [(documentation link)](https://docs.ros.org/en/noetic/api/geometry_msgs/html/msg/TransformStamped.html). Similarly to the Pose message used in earlier examples, the main transform component consists of translation and rotation. Additionally, a `Header` [(documentation link)](https://docs.ros.org/en/noetic/api/std_msgs/html/msg/Header.html) message provides timing information and the transformation's reference frame.

In ROS, transforms are referenced using a string called *frame_id*, representing a frame's name. All transforms in the ROS system form a tree, which can be visualised using *tf2_tools* [(link to tutorial)](https://wiki.ros.org/tf2/Tutorials/Introduction%20to%20tf2).

**In order to be successfully published to ROS, a transform must be valid. This means, that at least the following message fields must be set:**

* `transform.header.stamp`: Timestamp when the transform occured. Usually set to the current time using `rospy.Time.now()`
* `transform.header.frame_id`: The transform's reference frame. Transforms are defined *frame_id $\to$ child_frame_id*
* `transform.child_frame_id`: The transform's child frame
* (`transform.transform.rotation.w`: The transform's rotation must be a valid quaternion. Since all values in the transform are initialised with 0, the easiest way to achieve a valid rotation is to set the w component to 0. This will result in no rotation.)

A `tf2_ros.TransformBroadcaster` object is created and allows publishing of transforms using the the `sendTransform` method. **In order to enable publishing of transforms, a ROS node has to be initialised before creating a TransformBroadcaster or TransformListener object.**

For filling in the transform of the TransformStamped message, please refer to the section *Conversion to/from ROS Pose*.

In [1]:
from geometry_msgs.msg import TransformStamped
import tf2_ros
import rospy
import time

rospy.init_node("basic_listener") # Node initialisation required for comunication with ROS

# Create transform
transform = TransformStamped()

# Add reference frame and set transform to occur at current time
transform.header.frame_id = "map"
transform.header.stamp = rospy.Time.now()

# Define child frame and the relative transform
# NOTE: The transform's rotation must be valid, therefore, w is set to 1 here
transform.child_frame_id = "my_new_transform"
transform.transform.translation.x = 5.
transform.transform.rotation.w = 1.

tfBuffer = tf2_ros.Buffer()
broadcaster = tf2_ros.TransformBroadcaster()

# Wait for buffer to be initialised
time.sleep(4)

broadcaster.sendTransform(transform)

To visualise this code cell's output, open two terminals and run `roscore` and `rostopic echo /tf` respectively before executing the code cell. The second terminal will show the published transform on the */tf* topic used internally by ROS to keep track of transforms.

## Listening to Transforms <a class="anchor" id="tflistener"></a>

Transformations can be read by creating a `tf2_ros.TransformListener` object from a `tf2_ros.Buffer`. This object contains the `lookup_transform` method, which lets users read the transform between two frames in a given time interval. The following example code waits indefinetly for a transformation from the map to the robot coordinate system to become available.

The method's documentation can be found at [this link](https://docs.ros.org/en/indigo/api/tf2_ros/html/c++/classtf2__ros_1_1Buffer.html#acabbd72cae8f49fb3b6ede3be7e34c55). The main arguments we provide are the source and target frames the transform occurs between as well as a timeout of ten seconds that the method waits for the the transform to become available (`rospy.Duration(10.0)`). Furthermore, we specify the time in the ROS system of when the transform has occured. A time of zero indicates, that we want the latest transform (`rospy.Time(0)`). 

In [2]:
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)

while True:
    try:
        transform = tfBuffer.lookup_transform('map', 'my_new_transform', rospy.Time(0), rospy.Duration(10.0))
        break
    except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException):
        print("Waiting for transform took longer than 10 sec")
        continue

print("Transform from /map to /my_new_transform: \n{}".format(transform))

Transform from /map to /my_new_transform: 
header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: "map"
child_frame_id: "my_new_transform"
transform: 
  translation: 
    x: 0.0
    y: 0.0
    z: 0.0
  rotation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0


To test this code cell, open two terminals and run `roscore` and the following command respectively: 

```bash
rostopic pub /tf tf2_msgs/TFMessage "transforms:
- header:
    seq: 0
    stamp:
      secs: 0
      nsecs: 0
    frame_id: 'map'
  child_frame_id: 'my_new_transform'
  transform:
    translation:
      x: 0.0
      y: 0.0
      z: 0.0
    rotation:
      x: 0.0
      y: 0.0
      z: 0.0
      w: 1.0"      
```

The second command continuously publishes a simple transform to ROS' *tf* topic. After having starte the two terminals, run the previous code cell to initialise a ROS node. Finally, this code cell can be run to listen to and print the published transform. 